In [1]:
import tensorflow as tf

In [2]:
file_url = 'https://github.com/PacktWorkshops/The-TensorFlow-Workshop/blob/master/Chapter08/dataset/fruits360.zip'

In [3]:
zip_dir = tf.keras.utils.get_file('fruits360.zip', origin=file_url, extract=True)

   8192/Unknown - 0s 0us/step

In [ ]:
import pathlib

In [ ]:
path = pathlib.Path(zip_dir).parent / 'fruits360_filtered'

In [ ]:
train_dir = path / 'Training'
validation_dir = path / 'Test'

In [ ]:
total_train = 11398
total_val = 4752

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

In [ ]:
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
batch_size = 32
img_height = 224
img_width = 224
channel = 3

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           target_size=(img_height, img_width))

Found 11398 images belonging to 120 classes.


In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(img_height, img_width))

Found 4752 images belonging to 120 classes.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
np.random.seed(8)
tf.random.set_seed(8)

In [ ]:
from tensorflow.keras.applications import NASNetMobile

In [ ]:
base_model = NASNetMobile(include_top=False, input_shape=(img_height, img_width, channel), weights='imagenet')

In [ ]:
for layer in base_model.layers[:700]:
  layer.trainable = False

In [ ]:
base_model.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_376 (Activation)     (None, 111, 111, 32) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

In [ ]:
model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(500, activation='relu'),
    layers.Dense(120, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])

In [ ]:
model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/5
712/712 [==============================] - 148s 207ms/step - loss: 2.7766 - accuracy: 0.4375 - val_loss: 3.0142 - val_accuracy: 0.4840
Epoch 2/5
712/712 [==============================] - 144s 202ms/step - loss: 0.5059 - accuracy: 0.8475 - val_loss: 3.0363 - val_accuracy: 0.5149
Epoch 3/5
712/712 [==============================] - 142s 200ms/step - loss: 0.2538 - accuracy: 0.9220 - val_loss: 0.9776 - val_accuracy: 0.7919
Epoch 4/5
712/712 [==============================] - 142s 200ms/step - loss: 0.2049 - accuracy: 0.9380 - val_loss: 0.8568 - val_accuracy: 0.8523
Epoch 5/5
712/712 [==============================] - 142s 199ms/step - loss: 0.1554 - accuracy: 0.9549 - val_loss: 0.8855 - val_accuracy: 0.8264
